### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [17]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("groqapikey")


In [3]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002B4CC701550>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002B4CC702120>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
from langchain_core.messages import HumanMessage,SystemMessage
messages=[
    SystemMessage(content="Translate the following English text to French"),
    HumanMessage(content="I love programming in Python.")
]
result=model.invoke(messages)
result

AIMessage(content="J'adore programmer en Python. \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 22, 'total_tokens': 33, 'completion_time': 0.02, 'prompt_time': 0.00132544, 'queue_time': 0.251521809, 'total_time': 0.02132544}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--819365f1-0c0e-4685-af25-dc503dfaf953-0', usage_metadata={'input_tokens': 22, 'output_tokens': 11, 'total_tokens': 33})

In [7]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

"J'adore programmer en Python. \n"

In [8]:
### Using LCEL- chain the components
chain=model | parser
chain.invoke(messages)

"J'adore programmer en Python.\n"

In [12]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate
generic_template="Translate the following into {language}:"
prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template), 
     ("user","{text}")]
)

In [15]:
result=prompt.invoke({"language":"French","text":"I Love Programming in Python"})
result.to_messages()

[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I Love Programming in Python', additional_kwargs={}, response_metadata={})]

In [16]:
##Chaining together components with LCEL
chain=prompt | model | parser
chain.invoke({"language":"French","text":"I Love Programming in Python"})

"J'aime programmer en Python.\n"